In [4]:
import ttn
from ipyleaflet import Map, basemaps


app_id = 'befinitiv-testnode'
access_key = 'ttn-account-****'

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, zoom=2)

def uplink_callback(msg, client):
  print('Received uplink from ', msg.dev_id)
  print(msg)
      
handler = ttn.HandlerClient(app_id, access_key)
mqtt_client = handler.data()
mqtt_client.set_uplink_callback(uplink_callback)
mqtt_client.connect()

Received uplink from  testnode
MSG(app_id='befinitiv-testnode', dev_id='testnode', hardware_serial='009B45E4337B4515', port=1, counter=0, payload_raw='EjQ=', metadata=MSG(time='2020-03-15T21:21:43.2749658Z'))
Received uplink from  testnode
MSG(app_id='befinitiv-testnode', dev_id='testnode', hardware_serial='009B45E4337B4515', port=1, counter=0, is_retry=True, payload_raw='8AEcAQ==', metadata=MSG(time='2020-03-15T21:22:28.028471756Z', frequency=868.1, modulation='LORA', data_rate='SF10BW125', airtime=329728000, coding_rate='4/5', gateways=[MSG(gtw_id='eui-fcc23dfffe0e316a', timestamp=593687748, time='', channel=0, rssi=-95, snr=11.8, rf_chain=0)]))


In [2]:
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…